# Cleaning the provinces

In [52]:
# Import packages that are used
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
import seaborn as sns

# To ensure that axis-texts in plots are not cut off
plt.tight_layout()

<Figure size 432x288 with 0 Axes>

In [53]:
df = pd.read_csv('Data/winemag-data-130k-v2.csv', sep = ',')
france = df[df.country.str.contains('France', na=False)]
france.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
7,7,France,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach
9,9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
11,11,France,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer


In [71]:
# Drop unnecessary columns
wines = france.drop(['Unnamed: 0', 'country', 'region_2', 'taster_name','taster_twitter_handle'], axis=1)\
    .rename(columns={'region_1':'region'}).reset_index(drop = True)

# Extract year from title
wines['year'] = wines['title'].str.extract('(\d{4})')
wines.head(3)

,description,designation,points,price,province,region,title,variety,winery,year
0,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012
1,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012
2,"This is a dry wine, very spicy, with a tight, ...",NaN,87,30.0,Alsace,Alsace,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer,2012


In [72]:
wines.province.unique()

array(['Alsace', 'Beaujolais', 'Bordeaux', 'Champagne', 'Burgundy',
       'France Other', 'Southwest France', 'Rhône Valley',
       'Languedoc-Roussillon', 'Provence', 'Loire Valley'], dtype=object)

### Replacing Southwest France with Midi-Pyrénées or Aquitaine

In [73]:
wines[wines.province == 'Southwest France'].region.unique()

array(['Cahors', 'Madiran', 'Vin de Pays des Côtes de Gascogne',
       'Côtes de Gascogne', 'Côtes du Lot', 'Gaillac', 'Montravel',
       'Monbazillac', 'Gers', 'Fronton', 'Bergerac', 'Jurançon',
       'Bergerac Sec', 'Buzet', 'Irrouléguy', 'Côtes du Marmandais',
       'Pécharmant', 'Vin de Pays du Comté Tolosan', nan, 'Côtes du Tarn',
       'Côtes de Bergerac', 'Pacherenc du Vic Bilh', 'Jurançon Sec',
       'Côtes de Duras', 'Vin de Pays du Lot', 'Marcillac', 'Saint-Mont',
       'Saussignac', 'Côtes du Frontonnais', 'Brulhois', 'Landes',
       'Périgord', 'Bergerac Rosé', 'Lot', 'Béarn', 'Gaillac Doux',
       'Côtes de Saint-Mont', 'Côtes de Montravel'], dtype=object)

In [74]:
midi_pyrenees = ['Cahors', 'Madiran', 'Côtes de Gascogne', 'Côtes du Lot', 'Gaillac Doux', 
                 'Gers', 'Fronton', 'Côtes du Tarn', 'Pacherenc du Vic Bilh', 'Vin de Pays du Lot', 
                 'Marcillac', 'Saint-Mont', 'Côtes du Frontonnais', 'Brulhois', 'Lot', 
                 'Côtes de Saint-Mont', 'Vin de Pays des Côtes de Gascogne']
aquitaine = ['Montravel', 'Monbazillac', 'Bergerac', 'Bergerac Sec', 'Bergerac Rosé', 
             'Jurançon', 'Jurançon Sec', 'Buzet', 'Irrouléguy', 
             'Côtes du Marmandais', 'Pécharmant', 'Côtes de Bergerac', 'Côtes de Duras', 
             'Saussignac', 'Landes', 'Périgord', 'Béarn', 'Gaillac', 'Côtes de Montravel']

In [75]:
wines.loc[wines.region.isin(midi_pyrenees), 'province'] = 'Midi Pyrénées'
wines.loc[wines.region.isin(aquitaine), 'province'] = 'Aquitaine'

In [76]:
wines[wines.province == 'Southwest France'].region.unique()

array(['Vin de Pays du Comté Tolosan', nan], dtype=object)

### Replacing France Other

In [77]:
wines[wines.province == 'France Other'].region.unique()

array(['Vin de France', 'Arbois', 'Patrimonio', 'Crémant de Jura',
       'Vin de Liqueur', 'Vin de Savoie', 'Mediterranée', 'Vin Mousseux',
       'Corse', 'Ile de Beauté', 'Roussette de Savoie', 'Côtes du Jura',
       'Savoie', 'Corse Porto Vecchio', 'Vin de Table Francais',
       'Atlantique', 'Chignin-Bergeron', nan, 'France', 'Vin Pétillant',
       'Château-Chalon', 'Vin de Pays des Portes de Méditerranée',
       'Vin de Pays de la Méditerranée', "Vin de Pays de l'Ile de Beauté",
       'Chignin', 'Côtes de Forez', 'Charentais', 'Vin de Pays de France',
       'Apremont'], dtype=object)

In [78]:
corse = ['Corse', 'Patrimonio', 'Vin de Pays de l\'Ile de Beauté', 'Ile de Beauté', 'Corse Porto Vecchio']
rhone = ['Savoie', 'Vin de Savoie', 'Roussette de Savoie', 'Chignin', 'Côtes de Forez', 'Chignin-Bergeron']
provence = ['Vin de Pays des Portes de Méditerranée']
poitou_charantes = ['Charentais']
franche_comte = ['Château-Chalon', 'Crémant de Jura', 'Côtes du Jura', 'Arbois']
loire = ['Apremont']

In [79]:
wines.loc[wines.region.isin(corse), 'province'] = 'Corse'
wines.loc[wines.region.isin(rhone), 'province'] = 'Rhône Valley'
wines.loc[wines.region.isin(provence), 'province'] = 'Provence'
wines.loc[wines.region.isin(poitou_charantes), 'province'] = 'Poitou-Charentes'
wines.loc[wines.region.isin(franche_comte), 'province'] = 'Franche-Comté'
wines.loc[wines.region.isin(loire), 'province'] = 'Loire Valley'

In [80]:
wines[wines.province == 'France Other'].region.unique()

array(['Vin de France', 'Vin de Liqueur', 'Mediterranée', 'Vin Mousseux',
       'Vin de Table Francais', 'Atlantique', nan, 'France',
       'Vin Pétillant', 'Vin de Pays de la Méditerranée',
       'Vin de Pays de France'], dtype=object)

### Beaujolais = Brugundy & Bordeaux = Aquitaine

In [81]:
wines.loc[wines.province == 'Beaujolais', 'province'] = 'Burgundy'
wines.loc[wines.province == 'Bordeaux', 'province'] = 'Aquitaine'

In [82]:
wines.province.unique()

array(['Alsace', 'Burgundy', 'Aquitaine', 'Champagne', 'France Other',
       'Midi Pyrénées', 'Rhône Valley', 'Franche-Comté',
       'Languedoc-Roussillon', 'Provence', 'Loire Valley', 'Corse',
       'Southwest France', 'Poitou-Charentes'], dtype=object)

### Drop irrelevant provinces

In [83]:
irr_prov = ['Southwest France', 'France Other']
wines = wines[np.logical_not(wines.province.isin(irr_prov))]

### Reame provinces to correct names

In [84]:
wines.loc[wines.province == 'Burgundy', 'province'] = 'Bourgogne'
wines.loc[wines.province == 'Champagne', 'province'] = 'Champagne-Ardenne'
wines.loc[wines.province == 'Midi Pyrénées', 'province'] = 'Midi-Pyrénées'
wines.loc[wines.province == 'Loire Valley', 'province'] = 'Pays-de-la-Loire'
wines.loc[wines.province == 'Provence', 'province'] = 'Provence-Alpes-Côtes d\'Azur'
wines.loc[wines.province == 'Rhône Valley', 'province'] = 'Rhône-Alpes'

In [85]:
wines.province.unique()

array(['Alsace', 'Bourgogne', 'Aquitaine', 'Champagne-Ardenne',
       'Midi-Pyrénées', 'Rhône-Alpes', 'Franche-Comté',
       'Languedoc-Roussillon', "Provence-Alpes-Côtes d'Azur",
       'Pays-de-la-Loire', 'Corse', 'Poitou-Charentes'], dtype=object)

### Change points format

In [86]:
wines.points = wines.points.div(10).round(1)

In [87]:
wines.price = wines.price.round(1)

In [92]:
wines.head(3)

,description,designation,points,price,province,region,title,variety,winery,year
0,This dry and restrained wine offers spice in p...,NaN,8.7,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012
1,This has great depth of flavor with its fresh ...,Les Natures,8.7,27.0,Alsace,Alsace,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam,2012
2,"This is a dry wine, very spicy, with a tight, ...",NaN,8.7,30.0,Alsace,Alsace,Leon Beyer 2012 Gewurztraminer (Alsace),Gewürztraminer,Leon Beyer,2012


### Export the dataset

In [16]:
# Export the wines dataset to Firebase
wines.to_json(path_or_buf='Data/raw.json', orient='records')

# Grouping the data

In [18]:
wines.head(1)

,description,designation,points,price,province,region,title,variety,winery,year
0,This dry and restrained wine offers spice in p...,NaN,87,24.0,Alsace,Alsace,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012


### Data for home page

In [93]:
province_info = wines.groupby(['province']).agg(regions=('region', 'unique'), 
                                                 n_regions=('region', pd.Series.nunique),
                                                 n_varieties=('variety', pd.Series.nunique), 
                                                 avg_price=('price', 'mean'), 
                                                 avg_points=('points', 'mean')).reset_index()

,province,regions,n_regions,n_varieties,avg_price,avg_points
0,Alsace,"[Alsace, Crémant d'Alsace]",2,20,30.376805,8.937213
1,Aquitaine,"[Bordeaux Blanc, Saint-Émilion, Saint-Estèphe,...",63,35,37.375628,8.864757
2,Bourgogne,"[Beaujolais-Villages, Beaujolais, Brouilly, Ch...",130,17,60.203188,8.923388


In [94]:
province_info.avg_points = province_info.avg_points.round(1)
province_info.avg_price = province_info.avg_price.round(1)
province_info.head(3)

,province,regions,n_regions,n_varieties,avg_price,avg_points
0,Alsace,"[Alsace, Crémant d'Alsace]",2,20,30.4,8.9
1,Aquitaine,"[Bordeaux Blanc, Saint-Émilion, Saint-Estèphe,...",63,35,37.4,8.9
2,Bourgogne,"[Beaujolais-Villages, Beaujolais, Brouilly, Ch...",130,17,60.2,8.9


In [33]:
# Export province_info dataset
province_info.to_json(path_or_buf='Data/province_info.json', orient='records')

### Data for varieties page

In [110]:
prov_varieties = wines.groupby(['province', 'variety'])\
                        .agg(regions=('region', 'unique'), 
                             n_titles=('title', 'count'), 
                             avg_price=('price', 'mean'), 
                             avg_points=('points', 'mean'))\
                        .sort_values(['province', 'n_titles'], ascending=[True, False])\
                        .reset_index()

In [112]:
prov_varieties.avg_points = prov_varieties.avg_points.round(1)
prov_varieties.avg_price = prov_varieties.avg_price.round(1)
prov_varieties.head(3)

,province,variety,regions,n_titles,avg_price,avg_points
0,Alsace,Riesling,"[Alsace, Crémant d'Alsace]",718,32.4,9.0
1,Alsace,Gewürztraminer,[Alsace],476,34.3,8.9
2,Alsace,Pinot Gris,"[Alsace, Crémant d'Alsace]",458,33.4,9.0


In [25]:
# Export prov_varieties dataset
prov_varieties.to_json(path_or_buf='Data/prov_varieties.json', orient='records')

### Data for search engine page

In [115]:
varieties = wines.groupby(['variety'])\
                        .agg(provinces=('province', 'unique'), 
                             n_titles=('title', 'count'), 
                             avg_price=('price', 'mean'), 
                             avg_points=('points', 'mean'))\
                        .sort_values(['n_titles'], ascending=[False])\
                        .reset_index()

In [116]:
varieties.avg_points = varieties.avg_points.round(1)
varieties.avg_price = varieties.avg_price.round(1)
varieties.head(3)

,variety,provinces,n_titles,avg_price,avg_points
0,Bordeaux-style Red Blend,"[Aquitaine, Midi-Pyrénées, Provence-Alpes-Côte...",4724,40.9,8.9
1,Chardonnay,"[Bourgogne, Champagne-Ardenne, Languedoc-Rouss...",2769,58.1,8.9
2,Pinot Noir,"[Bourgogne, Alsace, Pays-de-la-Loire, Rhône-Al...",1928,83.5,9.0


In [ ]:
# Export varieties dataset
varieties.to_json(path_or_buf='Data/varieties.json', orient='records')

### Look at the data

In [27]:
wines.province.value_counts()

Aquitaine               6370
Burgundy                5024
Alsace                  2440
Loire Valley            1857
Champagne               1613
Provence                1349
Rhône Valley            1139
Midi Pyrénées           1036
Languedoc-Roussillon     613
Corse                    113
Franche-Comté             73
Poitou-Charentes           2
Name: province, dtype: int64

In [28]:
provinces = wines.groupby(['province', 'region']).agg({'variety':'count'}).reset_index()
provinces.groupby(['province']).agg({'region':'count'})

,region
province,
Alsace,2
Aquitaine,63
Burgundy,130
Champagne,1
Corse,5
Franche-Comté,4
Languedoc-Roussillon,50
Loire Valley,48
Midi Pyrénées,17


We have to many regions for our visualization. Look for example at Brugundy. We just want a simple visualization for wine amateurs and aren't interested in tiny regions.

### What happens when cleaning varieties & regions a little?

We only want to study the most common varieties as the visualization is meant for amateurs.

In [29]:
varieties = wines.variety.value_counts()

In [30]:
# Create dataset with only these varietes. 
condition = varieties > 100
wanted_varieties = varieties[condition]

wines_lim = wines[wines.variety.isin(wanted_varieties.index)]

In [31]:
wines_lim.province.value_counts()

Aquitaine               6239
Burgundy                4985
Alsace                  2283
Loire Valley            1797
Champagne               1594
Provence                1215
Rhône Valley             968
Midi Pyrénées            709
Languedoc-Roussillon     502
Corse                     81
Franche-Comté             45
Poitou-Charentes           2
Name: province, dtype: int64

We also want to study the most common regions as the visualization is for amateurs.

In [32]:
regions = wines_lim.region.value_counts()

In [33]:
# Create dataset with only these varietes. 
condition = regions > 8
wanted_regions = regions[condition]

wines_lim = wines_lim[wines_lim.region.isin(wanted_regions.index)]

In [34]:
wines_lim.province.value_counts()

Aquitaine               6153
Burgundy                4774
Alsace                  2283
Loire Valley            1739
Champagne               1594
Provence                1193
Rhône Valley             897
Midi Pyrénées            680
Languedoc-Roussillon     408
Corse                     75
Franche-Comté             45
Name: province, dtype: int64

We still have a good representation of provinces when doing so.

In [35]:
provinces = wines_lim.groupby(['province', 'region']).agg({'variety':'count'}).reset_index()
provinces.groupby(['province']).agg({'region':'count'})

,region
province,
Alsace,2
Aquitaine,42
Burgundy,74
Champagne,1
Corse,3
Franche-Comté,3
Languedoc-Roussillon,13
Loire Valley,25
Midi Pyrénées,8


We now have a better representation for regions. We can clearly see which provinces are larger wine producers and we don't have as many regions for Burgundy.

### Extracting all province names

In [37]:
wines.province.unique()

array(['Alsace', 'Burgundy', 'Aquitaine', 'Champagne', 'Midi Pyrénées',
       'Rhône Valley', 'Franche-Comté', 'Languedoc-Roussillon',
       'Provence', 'Loire Valley', 'Corse', 'Poitou-Charentes'],
      dtype=object)

In [ ]:
#wines.loc[wines.region == 'Vin de Pays des Côtes de Gascogne', 'region'] = 'Côtes de Gascogne'
#wines.loc[wines.region == 'Vin de Pays du Lot', 'region'] = 'Lot'
#wines.loc[wines.region == 'Bergerac Sec', 'region'] = 'Bergerac'
#wines.loc[wines.region == 'Jurançon Sec', 'region'] = 'Jurançon'
#wines.loc[wines.region == 'Bergerac Rosé', 'region'] = 'Bergerac'
#wines.loc[wines.region == 'Gaillac Doux', 'region'] = 'Gaillac'

In [ ]:
#wines.loc[wines.region == 'Roussette de Savoie', 'region'] = 'Savoie'
#wines.loc[wines.region == 'Vin de Savoie', 'region'] = 'Savoie'
#wines.loc[wines.region == 'Crémant de Jura', 'region'] = 'Jura'